In [1]:

import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras import backend as K
from keras.callbacks import EarlyStopping
from keras.preprocessing.image import ImageDataGenerator

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

from PIL import Image

import numpy as np
import os

Using TensorFlow backend.
C:\Users\nur.yilmaz\AppData\Roaming\Python\Python37\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\nur.yilmaz\AppData\Roaming\Python\Python37\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\nur.yilmaz\AppData\Roaming\Python\Python37\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\nu

In [2]:
image_dir = os.getcwd() + '\\Train_data_resized\\'
X = []
y = []
for image_path in os.listdir(os.fsencode(image_dir)):
    for image in os.listdir(os.fsencode(image_dir + str(image_path).replace("b'", '').replace("'", ''))):
        img_dir = os.fsencode(image_dir + str(image_path).replace("b'", '').replace("'", '') + '\\' + str(image).replace("b'", '').replace("'", ''))
        img = Image.open(img_dir)
        if img.size == (128,128):
            I = np.array(img.getdata(),
                        np.uint8).reshape(49152 , 1)

            X.append(I.reshape(1 , 49152))
            y.append(str(image_path).replace("b'", '').replace("'", ''))
        
        else:
            break

In [ ]:
np.array(X).shape[0]

In [ ]:
K.image_data_format() 

In [3]:
 X = np.array(X).reshape(np.array(X).shape[0], 3, 128, 128)

In [ ]:
"""if K.image_data_format() == 'channels_last':
        X = np.array(X).reshape(np.array(X).shape[0], 3, 128, 128)
        input_shape = (3, 128, 128)
else:
        X = np.array(X).reshape(np.array(X).shape[0], 128, 128, 3)
        input_shape = (128, 128, 3)
"""

In [4]:
X = X/255

In [5]:
lb = LabelEncoder()

y = lb.fit_transform(y)
y = keras.utils.to_categorical(y,6) 




In [6]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)

In [7]:
model = Sequential()

In [8]:
model.add(Conv2D(64, (3, 3), padding='same',
                 input_shape=X_train.shape[1:]))
# add relu activation function
model.add(Activation('relu'))

model.add(Dropout(0.3))
# add layer with 3x3 filters to model
model.add(MaxPooling2D(pool_size=(3, 3), dim_ordering="th"))
# randomly close 0.10 of neurons to prevent overfitting during training
model.add(Dropout(0.10))




C:\Users\nur.yilmaz\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:8: UserWarning: Update your `MaxPooling2D` call to the Keras 2 API: `MaxPooling2D(pool_size=(3, 3), data_format="channels_first")`
  


In [9]:
model.add(Conv2D(32, (3, 3), data_format='channels_first'))
# add relu activation function
model.add(Activation('relu'))

model.add(Dropout(0.3))
# add 2x2 MaxPooling layer 
model.add(MaxPooling2D(pool_size=(2, 2)))
# randomly close 0.10 of neurons to prevent overfitting during training
model.add(Dropout(0.10))

In [10]:
# convert 2D images to 1D vector
model.add(Flatten())
# add to model 512 neurons
model.add(Dense(256))
# add relu activation function
model.add(Activation('relu'))
# randomly close 0.30 of neurons
model.add(Dropout(0.3))

model.add(Dense(6)) # num_classes = 6
#add "Softmax" function to calculate the probability of classes
model.add(Activation('softmax'))

In [11]:
opt = keras.optimizers.adam(lr=0.01, decay=1e-6)

In [12]:
early_stopping_monitor = EarlyStopping(patience=3)

In [13]:
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [14]:
model.fit(X_train, y_train,
              batch_size=128,
              epochs=15,
              validation_data=(X_test, y_test),
              verbose=1,
              shuffle=True,
              )

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where

Train on 56 samples, validate on 15 samples
Epoch 1/15
56/56 [==============================] - 1s 19ms/step - loss: 0.6041 - accuracy: 0.7857 - val_loss: 0.7584 - val_accuracy: 0.7556
Epoch 2/15
56/56 [==============================] - 0s 8ms/step - loss: 1.4045 - accuracy: 0.7738 - val_loss: 1.1951 - val_accuracy: 0.8222
Epoch 3/15
56/56 [==============================] - 0s 7ms/step - loss: 2.6941 - accuracy: 0.7917 - val_loss: 0.6869 - val_accuracy: 0.8222
Epoch 4/15
56/56 [==============================] - 0s 7ms/step - loss: 1.6759 - accuracy: 0.7917 - val_loss: 0.3847 - val_accuracy: 0.8333
Epoch 5/15
56/56 [==============================] - 0s 7ms/step - loss: 0.5910 - accuracy: 0.7857 - val_loss: 0.4299 - val_accuracy: 0.8333
Epoch 6/15
56/56 [==============================] - 0s 7ms/step - loss: 0.4550 - accuracy: 0.8333 - val_loss: 0.4473 - val_accuracy: 0.8333
Epoch 7/15
56/56 [==

In [15]:
scores = model.evaluate(X_test, y_test, verbose=1)
print('Test loss:', scores[0])
print('Test accuracy:', scores[1])

15/15 [==============================] - 0s 3ms/step
Test loss: 0.41899335384368896
Test accuracy: 0.8333333134651184


In [16]:
model.save("model.h5")

In [ ]:
import glob
from PIL import ImageOps
from PIL import Image
import numpy as np

img_dir = glob.glob('C://Users//**//Desktop//**//ImageCrawler - Deploy//Test_Data//*')
  
for file in img_dir :
    for image_dir in glob.glob(file + "//*"): 
        
        img = Image.open(image_dir)
        new_path = image_dir.replace('Test_Data', 'Test_Data_resized').replace('\\', '//')
        
        # Create each image from related directory via Image
        
        # Convert to RGB
        img = img.convert('RGB')
        # Resize image to the biggest dimension by making aspect ratio 1:1
        old_size = img.size
        max_size =  np.max(list(old_size))
        new_size = (max_size, max_size)
        # Create a canvas with white background, insert considered image to its center
        new_im = Image.new("RGB", new_size, 'white')
        new_im.paste(img, (int((new_size[0]-old_size[0])/2),
                      int((new_size[1]-old_size[1])/2)))
        # Autocontrast the image.
        auto_img = ImageOps.autocontrast(new_im)
        auto_img.thumbnail((128, 128))
        
        # Save the image to new path.
        auto_img.save(new_path)
        

In [17]:
import flask

import keras.models

from werkzeug.wrappers import Request, Response
from flask import Flask
from flask import request

from keras.applications.imagenet_utils import decode_predictions


In [18]:
app = Flask(__name__)

@app.route('/handle_request', methods=['GET', 'POST'])
def handle_request():
    
    
    img = request.args.get('path')
    
    img = Image.open(img)
    loaded_model = keras.models.load_model('model.h5')
    A = []
    
    I = np.array(img.getdata(),
                np.uint8).reshape(49152 , 1)
    A.append(I.reshape(1 , 49152))

    A = np.array(A).reshape(np.array(A).shape[0], 3, 128, 128)
    A = A/255

    preds = loaded_model.predict(A)
    
     
    y_classes = preds.argmax(axis=-1)
    
   
    
    return str(lb.inverse_transform(y_classes[0]))
    
    
    

  


In [ ]:
if __name__ == '__main__':
    from werkzeug.serving import run_simple
    run_simple('localhost', 9000, app)

 * Running on http://localhost:9000/ (Press CTRL+C to quit)
C:\Users\nur.yilmaz\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
127.0.0.1 - - [15/Oct/2019 14:15:35] "GET /handle_request?path=C:\389725.jpg HTTP/1.1" 200 -
